In [1]:
import pandas as pd
import numpy as np 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

In [2]:
train_df = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


In [3]:
x = train_df[["Pclass","Sex","Age","SibSp","Parch","Fare"]]
y  = train_df["Survived"]

In [4]:
y

0        1
1        0
2        0
3        0
4        1
        ..
99995    1
99996    0
99997    0
99998    0
99999    0
Name: Survived, Length: 100000, dtype: int64

In [5]:
le = preprocessing.LabelEncoder()
x["Sex"] = le.fit_transform(x["Sex"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
x["Age"] =x["Age"].fillna(x["Age"].mean()) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
x.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,99866.00000
mean,2.106910,0.56114,38.355472,0.397690,0.454560,43.92933
std,0.837727,0.49625,18.009589,0.862566,0.950076,69.58882
min,1.000000,0.00000,0.080000,0.000000,0.000000,0.68000
25%,1.000000,0.00000,25.000000,0.000000,0.000000,10.04000
50%,2.000000,1.00000,38.355472,0.000000,0.000000,24.46000
75%,3.000000,1.00000,53.000000,1.000000,1.000000,33.50000
max,3.000000,1.00000,87.000000,8.000000,9.000000,744.66000


In [8]:
x_train,x_val,y_train,y_val = train_test_split(x,y)

In [9]:
model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)

In [10]:
 model_trained = model.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:03:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
print(classification_report(y_val,model_trained.predict(x_val)))

              precision    recall  f1-score   support

           0       0.79      0.80      0.79     14339
           1       0.72      0.72      0.72     10661

    accuracy                           0.76     25000
   macro avg       0.76      0.76      0.76     25000
weighted avg       0.76      0.76      0.76     25000



In [12]:
final_model = model.fit(x,y)

[07:03:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [13]:
test_df = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [14]:
test_df_clean = test_df[["Pclass","Sex","Age","SibSp","Parch","Fare"]]
le = preprocessing.LabelEncoder()
test_df_clean["Sex"] = le.fit_transform(test_df_clean["Sex"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
preds = final_model.predict(test_df_clean)

In [16]:
preds

array([0, 0, 1, ..., 0, 1, 1])

In [17]:
submit = pd.DataFrame({"PassengerId":test_df["PassengerId"],"Survived":preds})

In [18]:
submit

,PassengerId,Survived
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,1
...,...,...
99995,199995,0
99996,199996,0
99997,199997,0
99998,199998,1


In [19]:
submit.to_csv("submission.csv",index = False)